In [1]:
import arcpy
import pandas as pd
import psycopg2
from psycopg2 import sql
import numpy as np
import os
import folium

In [61]:
cwd = os.getcwd() # This is a global variable for where the notebook is (must change if running in arcpro)

# Make it workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

arcpy.env.overwriteOutput = True # Overwrite layers is okay

save_path_csv = os.path.join(cwd, '..', '..', 'data')
save_path_gdb = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')
historic_path = os.path.join(cwd, '..', '..', 'data', 'purpleair_historic.csv')
station_path = os.path.join(cwd, '..', '..', 'data', 'purpleair_stations.csv')
boundary_path = os.path.join(cwd, '..', '..', 'data', 'mpls_boundary.geojson')

In [64]:
output_fc = 'mpls_boundary'
output_fc_path = os.path.join(save_path_gdb, output_fc)

# Convert the GeoJSON file to a feature class
arcpy.JSONToFeatures_conversion(boundary_path, output_fc)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\mpls_boundary'>

In [20]:
df = pd.read_csv(historic_path)

# Convert the 'timestamp' column to a datetime object
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Filter the rows that have a timestamp of 4/1/23
date_filter = df['timestamp'].dt.date == pd.to_datetime('4/1/23').date()
filtered_df = df[date_filter]

filtered_df.to_csv('historic_april1.csv', index=False)
historic_filtered_path = os.path.join(cwd, '..', '..', 'data', 'historic_april1.csv')

In [55]:
# Load the first CSV file into a DataFrame
df1 = pd.read_csv(station_path)

# Load the second CSV file into a DataFrame
df2 = pd.read_csv(historic_filtered_path)

# Merge the two DataFrames together on a common column
merged_df = pd.merge(df1, df2, on='sensor_index')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)
merged_path = os.path.join(cwd, '..', '..', 'data', 'merged_file.csv')

In [56]:
# Set output point feature class name and path
output_point_fc = "stations_XYTableToPoint"
output_point_path = os.path.join(save_path_gdb, output_point_fc)

# Specify x,y fields
x_field = "longitude"
y_field = "latitude"

# Use XYTableToPoint tool to create point features
station_points = arcpy.management.XYTableToPoint(merged_path, output_point_path, x_field, y_field)

In [50]:
'''
output_table = "joined_table"
arcpy.management.AddJoin(
    in_layer_or_view=station_points,
    in_field="sensor_index",
    join_table=historic_filtered_path,
    join_field="sensor_index",
    join_type="KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS"
)

# Export the joined layer to a new table
historic_apr1 = arcpy.management.CopyRows("stations_XYTableToPoint", output_table)
'''

## PM 2.5 Interpolation

### Kriging PM 2.5

In [57]:
out_kriging_raster = "Kriging_Apr1"
arcpy.ddd.Kriging(
    in_point_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_surface_raster=out_kriging_raster,
    semiVariogram_props="Spherical # # # #",
    cell_size=0.000932919999999967,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Kriging_Apr1'>

### Clip Kriging

In [68]:
out_clip_kriging_raster = "Kriging_clip_Apr1"
arcpy.management.Clip(
    in_raster=out_kriging_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_kriging_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Kriging_clip_Apr1'>

### IDW PM 2.5

In [69]:
out_idw_raster = "IDW_Apr1"
arcpy.ga.IDW(
    in_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_ga_layer=None,
    out_raster=out_idw_raster,
    cell_size=0.000932919999999967,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result ''>

### Clip IDW

In [70]:
out_clip_idw_raster = "IDW_clip_Apr1"
arcpy.management.Clip(
    in_raster=out_idw_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_idw_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\IDW_clip_Apr1'>

### Spline PM 2.5

In [75]:
out_spline_raster = arcpy.sa.Spline(
    in_point_features="stations_XYTableToPoint",
    z_field="pm2_5",
    cell_size=0.000932919999999967,
    spline_type="REGULARIZED",
    weight=0.1,
    number_points=12
)
save_path_spline = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb', 'Spline_Apr1')
out_spline_raster.save(save_path_spline)

### Clip Spline

In [76]:
out_clip_spline_raster = "Spline_clip_Apr1"
arcpy.management.Clip(
    in_raster=out_spline_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_spline_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Spline_clip_Apr1'>

## Temperature Interpolation

### Kriging Temperature

In [84]:
out_kriging_temp_raster = "Kriging_Temp_Apr1"
arcpy.ddd.Kriging(
    in_point_features="stations_XYTableToPoint",
    z_field="temperature",
    out_surface_raster=out_kriging_temp_raster,
    semiVariogram_props="Spherical # # # #",
    cell_size=0.000932919999999967,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Kriging_Temp_Apr1'>

### Clip Kriging Temp

In [85]:
out_clip_kriging_temp_raster = "Kriging_clip_temp_Apr1"
arcpy.management.Clip(
    in_raster=out_kriging_temp_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_kriging_temp_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Kriging_clip_temp_Apr1'>

### IDW Temp

In [82]:
out_idw_temp_raster = "IDW_Temp_Apr1"
arcpy.ga.IDW(
    in_features="stations_XYTableToPoint",
    z_field="temperature",
    out_ga_layer=None,
    out_raster=out_idw_temp_raster,
    cell_size=0.000932919999999967,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result ''>

### Clip IDW

In [83]:
out_clip_idw_temp_raster = "IDW_clip_temp_Apr1"
arcpy.management.Clip(
    in_raster=out_idw_temp_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_idw_temp_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\IDW_clip_temp_Apr1'>

### Spline Temp

In [86]:
out_spline_temp_raster = arcpy.sa.Spline(
    in_point_features="stations_XYTableToPoint",
    z_field="temperature",
    cell_size=0.000932919999999967,
    spline_type="REGULARIZED",
    weight=0.1,
    number_points=12
)
save_path_temp_spline = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb', 'Spline_Temp_Apr1')
out_spline_temp_raster.save(save_path_temp_spline)

### Clip Spline

In [87]:
out_clip_spline_temp_raster = "Spline_clip_temp_Apr1"
arcpy.management.Clip(
    in_raster=out_spline_temp_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_spline_temp_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Spline_clip_temp_Apr1'>

## Cross Validation

### Kriging PM 2.5

In [89]:
stations = pd.read_csv("merged_file.csv")

# Define the coordinates and values arrays
X = stations[['latitude', 'longitude']].values
z = stations['pm2_5'].values

# Define the leave-one-out cross-validation object
n_points = X.shape[0]
loo = [(np.delete(np.arange(n_points), i), i) for i in range(n_points)]

# Define an array to hold the cross-validation errors
errors = []

# Loop over each point in the DataFrame and leave it out in turn
for train_index, test_index in loo:
    # Leave out the i-th point
    X_train, X_test = X[train_index], X[test_index]
    z_train, z_test = z[train_index], z[test_index]

    # Define the kriging model with the desired parameters
    kModel = arcpy.sa.KrigingModelOrdinary("LINEAR")

    # Fit the kriging model to the training data
    kriging = arcpy.sa.Kriging(
        in_point_features=X_train,
        z_field="pm2_5",
        kriging_model=kModel,
        cell_size=0.000932919999999967
    )

    # Predict the value at the left-out point
    z_pred = kriging.predict(X_test)[0]

    # Compute the absolute error between the predicted and true values
    error = abs(z_pred - z_test)

    # Record the error in the array
    errors.append(error)

# Compute the mean absolute error over all the left-out points
mae = np.mean(errors)

# Print the mean absolute error
print(f"Mean Absolute Error: {mae:.3f}")

RuntimeError: Object: Error in executing tool

### IDW PM 2.5